In [ ]:
import pyodbc
import pandas as pd

# Database connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'

# Establish connection and fetch data
with pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
) as conn:
    query = "SELECT * FROM dbo.DAILY_RECORDS WHERE STATION_ID = 10987"
    df = pd.read_sql_query(query, conn)

# Convert from wide to long format
df_long = df.melt(id_vars=['STATION_ID', 'RECORD_YEAR'], 
                  var_name='Day', value_name='Discharge (CFS)')

# Extract and format the 'Day' column
df_long['Day'] = df_long['Day'].str.extract(r'(RV_\d{4})')  # Keep the RV_XXXX format
df_long['Day_Number'] = df_long['Day'].str[3:].astype(int)  # Extract numeric day (0101 -> 101)

# Create a proper 'Date' column
df_long['Date'] = pd.to_datetime(df_long['RECORD_YEAR'].astype(str) + df_long['Day'].str[3:], 
                                 format='%Y%m%d', errors='coerce')

# Convert Discharge to numeric
df_long['Discharge (CFS)'] = pd.to_numeric(df_long['Discharge (CFS)'], errors='coerce')

# 🔹 Ensure correct sorting by year and day
df_long = df_long.sort_values(by=['RECORD_YEAR', 'Day_Number'])

# Select final columns
df_long = df_long[['STATION_ID', 'RECORD_YEAR', 'Day', 'Date', 'Discharge (CFS)']]

# 🔹 Save the correctly sorted data to CSV
output_filename = "station_10987_rearranged.csv"
df_long.to_csv(output_filename, index=False, date_format='%m/%d/%Y')  # Ensures proper date format

# 🔹 Print first 10 rows to confirm correct sorting
print(df_long.head(10))

print(f"\n✅ Data has been saved correctly in {output_filename}")